In [5]:
!pip install numpy
!pip install scikit-surprise
!pip install pandas

In [0]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random
import pickle

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [0]:
file_path = './Data/preprocessed_user_item_rating.csv'

In [11]:
if not (os.path.exists(file_path)):
  raise ValueError('El archivo preprocessed_user_item_rating.csv no fue encontrado en el path')
else:
  print("El archivo ha sido cargado")

El archivo ha sido cargado


In [12]:
ratings=pd.read_csv(file_path, sep = ',', header=0, names = [ 'userid', 'artist-name', 'rating' ] )
ratings = ratings.loc[:,['userid', 'artist-name','rating']]
ratings

,userid,artist-name,rating
0,user_001000,Wilco,5.0
1,user_001000,Radiohead,4.9
2,user_001000,Animal Collective,4.7
3,user_001000,Girl Talk,4.6
4,user_001000,Aesop Rock,4.2
...,...,...,...
896880,user_000001,Jamie Lidell,0.2
896881,user_000001,Nick Holmes,0.2
896882,user_000001,Nuyorican Soul,0.2
896883,user_000001,The Birthday,0.2


In [13]:
ratings.rating.describe()

count    167494.000000
mean          0.635761
std           0.737520
min           0.200000
25%           0.200000
50%           0.400000
75%           0.700000
max           5.000000
Name: rating, dtype: float64

# Creacion sistema de recomendacion

In [0]:
reader = Reader( rating_scale = ( 0, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'userid', 'artist-name', 'rating' ] ], reader )

In [0]:
trainset, testset=  train_test_split(surprise_dataset, test_size=.2)

### Modelo basado en distancias coseno

In [0]:
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud item-item
               }
algo = KNNBasic(k=50, min_k=10, sim_options=sim_options)

In [17]:
predictions = algo.fit(trainset).test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [20]:
pickle.dump( predictions, open( "./Data/predictions_uu_cosine.p", "wb" ) )

FileNotFoundError: ignored

### Modelo basado en índice de Jaccard

In [23]:
ratings_jaccard=ratings.groupby('userid')['artist-name'].apply(lambda x: ','.join(x)).tolist()
ratings_jaccard

["坂本龍一,Underworld,Cornelius,Gilles Peterson,Björk,Plaid,The Cinematic Orchestra,Röyksopp,Clark,Minilogue,Suzukiski,Towa Tei,Radiohead,Flying Lotus,Masomenos,Scsi-9,Kuniyuki Takahashi,Loco Dice,Boards Of Canada,London Elektricity,Herbert,Onra & Quetzal,矢野顕子,Jimpster,Zazen Boys,原田郁子,Squarepusher,高木正勝,The Black Dog,4Hero,Ian O'Brien,Steve Reich,Enya,Nitin Sawhney,Reel People,Richie Hawtin,Agustin Pereyra Lucena,Rasmus Faber,Milosh,Calm,Jazzanova,Janet Jackson,Third World Love,Bugge Wesseltoft,Woodblue,Zeep,Madlib,Recloose,Fleet Foxes,Afta-1,The Matthew Herbert Big Band,Jazzanova Feat. Paul Randolph,Pat Metheny Group,Hudson Mohawke,Ben Westbeech,I Am Robot And Proud,Plaid & Bob Jaroc,大樹,Part Time Heroes,Designed People,Burial,A Hundred Birds,The Ananda Project,Lism,Shugo Tokumaru,Rasmus Faber Feat. Dyanna Fearon,Sneaky Sound System,くるり,The Young Lovers,Lucky & Easy,Opiate,Ryukyudisko,Moby,吉澤はじめ,Mika,Starboard,The Verve,Dapayk Solo,Deadbeat,Erykah Badu,Miss Kittin & The Hacker,Rasmus Faber 

In [28]:
Lista=[]
for x in ratings_jaccard:
    lista=[]
    cont=0
    for y in ratings_jaccard:
        if x is not y:
            z = len(set(x).intersection(y)) / len(set(x).union(y))
            lista.append(str(z)+str(': ')+str(cont))
            cont+=1
    Lista.append(lista)
    
jaccard_df = pd.DataFrame(Lista)
jaccard_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990
0,0.5225225225225225: 0,0.5922330097087378: 1,0.5: 2,0.5784313725490197: 3,0.40789473684210525: 4,0.5132743362831859: 5,0.16326530612244897: 6,0.49107142857142855: 7,0.5643564356435643: 8,0.5700934579439252: 9,0.5544554455445545: 10,0.5648148148148148: 11,0.53: 12,0.5238095238095238: 13,0.5636363636363636: 14,0.5344827586206896: 15,0.45: 16,0.5047619047619047: 17,0.46078431372549017: 18,0.5700934579439252: 19,0.5673076923076923: 20,0.49514563106796117: 21,0.5728155339805825: 22,0.5081967213114754: 23,0.5333333333333333: 24,0.54: 25,0.6176470588235294: 26,0.54: 27,0.5339805825242718: 28,0.5614035087719298: 29,0.35353535353535354: 30,0.35135135135135137: 31,0.5225225225225225: 32,0.432258064516129: 33,0.4609375: 34,0.4580152671755725: 35,0.42276422764227645: 36,0.5: 37,0.5514018691588785: 38,0.5535714285714286: 39,...,0.5656565656565656: 951,0.5689655172413793: 952,0.3316831683168317: 953,0.5163934426229508: 954,0.5196078431372549: 955,0.48484848484848486: 956,0.5625: 957,0.5588235294117647: 958,0.5327102803738317: 959,0.5: 960,0.5526315789473685: 961,0.6: 962,0.4351145038167939: 963,0.5263157894736842: 964,0.5816326530612245: 965,0.5445544554455446: 966,0.5841584158415841: 967,0.5294117647058824: 968,0.5: 969,0.5784313725490197: 970,0.504950495049505: 971,0.5242718446601942: 972,0.5196078431372549: 973,0.5350877192982456: 974,0.37349397590361444: 975,0.5825242718446602: 976,0.5784313725490197: 977,0.5480769230769231: 978,0.5757575757575758: 979,0.46875: 980,0.464: 981,0.49038461538461536: 982,0.5045045045045045: 983,0.5686274509803921: 984,0.54: 985,0.49504950495049505: 986,0.3979591836734694: 987,0.5607476635514018: 988,0.4628099173553719: 989,0.5596330275229358: 990
1,0.5225225225225225: 0,0.7564102564102564: 1,0.7710843373493976: 2,0.7631578947368421: 3,0.4609375: 4,0.7142857142857143: 5,0.22535211267605634: 6,0.627906976744186: 7,0.7466666666666667: 8,0.7625: 9,0.7105263157894737: 10,0.6904761904761905: 11,0.726027397260274: 12,0.7733333333333333: 13,0.8354430379746836: 14,0.7159090909090909: 15,0.6619718309859155: 16,0.7236842105263158: 17,0.6486486486486487: 18,0.7625: 19,0.7894736842105263: 20,0.6493506493506493: 21,0.8493150684931506: 22,0.6354166666666666: 23,0.6962025316455697: 24,0.7397260273972602: 25,0.7468354430379747: 26,0.7397260273972602: 27,0.7236842105263158: 28,0.7764705882352941: 29,0.5070422535211268: 30,0.4180327868852459: 31,0.775: 32,0.4444444444444444: 33,0.6494845360824743: 34,0.5327102803738317: 35,0.5578947368421052: 36,0.684931506849315: 37,0.8051948051948052: 38,0.75: 39,...,0.7777777777777778: 951,0.648936170212766: 952,0.3988439306358382: 953,0.6989247311827957: 954,0.7297297297297297: 955,0.6901408450704225: 956,0.8048780487804879: 957,0.7368421052631579: 958,0.7792207792207793: 959,0.6708860759493671: 960,0.7647058823529411: 961,0.7272727272727273: 962,0.5046728971962616: 963,0.6704545454545454: 964,0.7534246575342466: 965,0.6973684210526315: 966,0.7051282051282052: 967,0.72: 968,0.6391752577319587: 969,0.7402597402597403: 970,0.7123287671232876: 971,0.6883116883116883: 972,0.7297297297297297: 973,0.7011494252873564: 974,0.4154929577464789: 975,0.7215189873417721: 976,0.7866666666666666: 977,0.6962025316455697: 978,0.7671232876712328: 979,0.7127659574468085: 980,0.6595744680851063: 981,0.5421686746987951: 982,0.7721518987341772: 983,0.7733333333333333: 984,0.7638888888888888: 985,0.6986301369863014: 986,0.5492957746478874: 987,0.8181818181818182: 988,0.6304347826086957: 989,0.7654320987654321: 990
2,0.5922330097087378: 0,0.7564102564102564: 1,0.6705882352941176: 2,0.8169014084507042: 3,0.4918032786885246: 4,0.7160493827160493: 5,0.24242424242424243: 6,0.6463414634146342: 7,0.8529411764705882: 8,0.7894736842105263: 9,0.81159

In [0]:
pickle.dump( Lista, open( "./Data/predictions_uu_jaccard.p", "wb" ) )
pickle_predictions_jaccard = pickle.load( open( "./Data/predictions_uu_jaccard.p", "rb" ) )

In [58]:
all_five_first_prediction_uu_jaccard=[]
for x in pickle_predictions_jaccard:
    a=np.sort(np.array(x))
    all_five_first_prediction_uu_jaccard.append(a[::-1][0:5].tolist())

pd.DataFrame(all_five_first_prediction_uu_jaccard)
all_five_first_prediction_uu_jaccard

[['0.6: 962', '0.6: 849', '0.6: 441', '0.6: 354', '0.6: 117'],
 ['0.9054054054054054: 433', '0.8: 726', '0.8: 486', '0.8: 407', '0.8: 239'],
 ['0.9: 282',
  '0.9142857142857143: 510',
  '0.9027777777777778: 720',
  '0.8: 988',
  '0.8: 98'],
 ['0.7: 957', '0.7: 91', '0.7: 842', '0.7: 79', '0.7: 638'],
 ['0.9117647058823529: 261',
  '0.9090909090909091: 242',
  '0.8: 977',
  '0.8: 900',
  '0.8: 88'],
 ['0.7441860465116279: 50',
  '0.7388059701492538: 587',
  '0.7238805970149254: 690',
  '0.7226890756302521: 581',
  '0.7213114754098361: 257'],
 ['0.8: 671', '0.7: 977', '0.7: 970', '0.7: 335', '0.7: 314'],
 ['0.41935483870967744: 558',
  '0.4074074074074074: 886',
  '0.3: 956',
  '0.3: 879',
  '0.3: 621'],
 ['0.7: 323',
  '0.7: 122',
  '0.725: 772',
  '0.7051282051282052: 568',
  '0.704225352112676: 610'],
 ['0.8: 951', '0.8: 827', '0.8: 820', '0.8: 668', '0.8: 485'],
 ['0.8: 67',
  '0.8533333333333334: 781',
  '0.84: 286',
  '0.8472222222222222: 549',
  '0.8421052631578947: 19'],
 ['0.8: 

Predicciones para el primer usuario:

In [59]:
s=[]
for x in all_five_first_prediction_uu_jaccard[0]:
  s.append(x.split(':')[1])

df=pd.DataFrame(s)
df

,0
0,962
1,849
2,441
3,354
4,117


### Modelo basado en correlación de Pearson

In [0]:
sim_options = {'name': 'pearson_baseline',
               'user_based': True,
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

In [61]:
predictions = algo.fit(trainset).test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [62]:
pickle.dump( predictions, open( "./Data/predictions_uu_pearson.p", "wb" ) )

FileNotFoundError: ignored

### Recomendaciones

In [0]:
pickle_predictions = pickle.load( open( "./Data/predictions_uu_pearson.p", "rb" ) )

In [0]:
#Predicciones para usuario user_001000
user_predictions=list(filter(lambda x: x[0]=='user_001000' and x[4]['was_impossible']==False,pickle_predictions))

In [0]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [0]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [0]:
#Se convierte a dataframe
labels = ['artist', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

### Recomendaciones usuario 001000

In [0]:
df_predictions

,artist,estimation
0,Crystal Castles,1.290774
1,Belle And Sebastian,1.191414
2,Cut Copy,1.119432
3,Bloc Party,1.053604
4,Of Montreal,0.965159
5,Massive Attack,0.935226
6,The Decemberists,0.927302
7,Ben Folds Five,0.916262
8,Morcheeba,0.869920
9,Final Fantasy,0.866152


In [0]:
accuracy.rmse( user_predictions, verbose = True )

RMSE: 0.4596


0.45959910359602174

In [0]:
algo.predict('user_001000','Kenny Burrell')

Prediction(uid='user_001000', iid='Kenny Burrell', r_ui=None, est=1.6393711339908208, details={'actual_k': 5, 'was_impossible': False})

In [0]:
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [0]:
user_inner_id = algo.trainset.to_inner_uid('user_001000')

In [0]:
# Retrieve inner ids of the nearest neighbors of Item.
user_neighbors = algo.get_neighbors(user_inner_id, k=10)
user_neighbors

[106, 15, 16, 28, 57, 90, 97, 151, 172, 223]

In [0]:
neighbors = (algo.trainset.to_raw_uid(rid)
                       for rid in user_neighbors)
for neighbor in neighbors:
    print(neighbor)

user_000947
user_000859
user_000586
user_000153
user_000651
user_000097
user_000425
user_000379
user_000468
user_000718
